In [1]:
import os,shutil
import glob
import numpy as np
import tensorflow as tf
import keras
from keras.models import *
from keras.layers import *
from keras.applications import DenseNet201
import tensorflow as tf, tensorflow.keras.backend as K
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.preprocessing.image import *
from keras.callbacks import EarlyStopping
from keras import regularizers,optimizers
from keras.callbacks import LearningRateScheduler
from keras import *


''' directory path '''
dataset_path = "../input/walk-or-run"

''' train images path'''
train_dir = os.path.join(dataset_path,'walk_or_run_train/train')

''' test images path'''
test_dir= os.path.join(dataset_path,'walk_or_run_test/test')

Using TensorFlow backend.


In [2]:
''' total number of classes '''
os.listdir(train_dir)

['walk', 'run']

In [3]:
batch_size = 16
epochs = 50

In [4]:
lr_start = 0.00001
lr_max = 0.00005 
lr_min = 0.00001
lr_rampup_epochs = 5
lr_sustain_epochs= 0
lr_exp_decay = 0.8

def learning_func(epoch):
    if epoch < lr_rampup_epochs:
        lr = (lr_max - lr_start) / lr_rampup_epochs * epoch + lr_start
    elif epoch < lr_rampup_epochs + lr_rampup_epochs:
        lr = lr_max
    else:
        lr = (lr_max - lr_min) * lr_exp_decay**(epoch - lr_rampup_epochs - lr_sustain_epochs) + lr_min
    return lr
    
''' Callback '''
lr_callback = keras.callbacks.LearningRateScheduler(learning_func(30), verbose = True)

In [5]:
''' Early Stopping '''
es = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 15)

In [6]:
def random_erasor(p=0.5, s_l=0.02, s_h=0.4, r_1=0.3, r_2=1/0.3, v_l=0, v_h=255, pixel_level=False):
    def eraser(inp_img):
        
        ''' image height, width, and channel '''
        img_height, img_width, img_channel = input_img.shape
        p_1 = np.random.rand()

        if p_1 > p:
            return inp_img
        
        while True:
            s = np.random.uniform(s_l, s_h) * img_height * img_width
            r = np.random.uniform(r_1, r_2)
            w = int(np.sqrt(s / r))
            h = int(np.sqrt(s * r))
            
            left = np.random.randint(0, img_width)
            top = np.random.randint(0, img_height)

            if left + w <= img_width and top + h <= img_height:
                break

        if pixel_level:
            c = np.random.uniform(v_l, v_h, (h, w, img_channel))
        else:
            c = np.random.uniform(v_l, v_h)

        inp_img[top:top + h, left:left + w, :] = c

        return inp_img

    return eraser

In [7]:
''' Data Augmentation '''
train_ds = ImageDataGenerator(rotation_range=30, width_shift_range=0.1, height_shift_range=0.1,
                                   shear_range=0.1,zoom_range=0.1,horizontal_flip=True, vertical_flip=False,
                                   validation_split=0.2, preprocessing_function = random_erasor(v_l=0, v_h=255))
test_ds = ImageDataGenerator()

''' flow from directory '''
test_ds = test_ds.flow_from_directory(train_dir, target_size=(224, 224), batch_size=16, class_mode='categorical',seed=2019, 
                                      color_mode='rgb')

val_ds = train.flow_from_directory(train_dir, target_size=(224, 224), batch_size=16, class_mode='categorical',
                                   color_mode='rgb', subset='validation')

test_ds = test_ds.flow_from_directory(test_dir, target_size=(224, 224), batch_size=16, class_mode='categorical', 
                                      color_mode='rgb')

Found 600 images belonging to 2 classes.
Found 119 images belonging to 2 classes.
Found 141 images belonging to 2 classes.


In [10]:
st = tf.distribute.get_strategy()

In [11]:
def model
    with st.scope():
        ''' DenseNet201 '''
        model = keras.Sequential([DenseNet201(input_shape=(224,224, 3),include_top=False,weights='imagenet'),
                keras.layers.GlobalAveragePooling2D(),
                ''' classificaion layer ''' 
                keras.layers.Dense(2, activation='softmax')])
        ''' compile the model '''
        model.compile(optimizer='adam', oss = 'categorical_crossentropy',metrics=['categorical_accuracy'])
        
    return model

In [12]:
model = model()

74842112/74836368 [==============================] - 3s 0us/step


In [13]:
steps_per_epochs = 600/batch_size

In [14]:
''' training '''
history = model.fit_generator(train_ds, epochs = 30, validation_data=test_ds,
            callbacks = [keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor = 0.5, patience =3, 
                                                           min_lr=0.00001, verbose=1, mode='min'),early_stopping])

Epoch 1/30
38/38 [==============================] - 172s 5s/step - loss: 0.8290 - categorical_accuracy: 0.5817 - val_loss: 960.6427 - val_categorical_accuracy: 0.4184
Epoch 2/30
38/38 [==============================] - 21s 551ms/step - loss: 0.6439 - categorical_accuracy: 0.6400 - val_loss: 27.3813 - val_categorical_accuracy: 0.4113
Epoch 3/30
38/38 [==============================] - 21s 551ms/step - loss: 0.5777 - categorical_accuracy: 0.7083 - val_loss: 3.8149 - val_categorical_accuracy: 0.5816
Epoch 4/30
38/38 [==============================] - 22s 571ms/step - loss: 0.6036 - categorical_accuracy: 0.6733 - val_loss: 1.3337 - val_categorical_accuracy: 0.5957
Epoch 5/30
38/38 [==============================] - 21s 551ms/step - loss: 0.6252 - categorical_accuracy: 0.6700 - val_loss: 42.7393 - val_categorical_accuracy: 0.4184
Epoch 6/30
38/38 [==============================] - 21s 545ms/step - loss: 0.6647 - categorical_accuracy: 0.6583 - val_loss: 25.2281 - val_categorical_accuracy: 0.

In [15]:
''' evaluating the model '''
results = model.evaluate_generator(test_generator)

In [16]:
print("test_loss:",results[0],"test acc:",results[1])

test_loss: 0.30673107504844666 test acc: 0.8085106611251831
